In [7]:
import numpy as np
from cpmpy import *

import timeit
from IPython.display import display, HTML # display some titles
import random

TODO: 

- add a description of MUS as an explanation for unsatisfiability
- Smallest MUS
- Adding preferences or weights to the constraints 

# Unsatisfiable int model

In [16]:
def unsat_int_model():
    # weights correspond to prefrences or difficutly of a constraint when
    # computing an OCUS
    # SMUS: assign unit weights 
    # weights = [1] * 10
    weights = [10, 10, 10, 1, 1, 40, 20, 20, 20, 1]

    x = intvar(-9, 9, name="x")
    y = intvar(-9, 9, name="y")

    m = Model(
        x < 0, 
        x < 1,
        x > 2,
        x == 4,
        y == 4, 
        (x + y > 0) | (y < 0),
        (y >= 0) | (x >= 0),
        (y < 0) | (x < 0),
        (y > 0) | (x < 0),
        AllDifferent(x,y) # invalid for musx_assum
    )
    assert (m.solve() is False)

    return m, weights

# SUDOKU  

In [9]:
def print_sudoku(grid):
    # Puzzel dimensions
    nrow = ncol = len(grid)
    n = int(nrow ** (1/2))

    out = ""    
    for r in range(0,nrow):
        for c in range(0,ncol):
            out += str(grid[r, c] if grid[r, c] > 0 else ' ')
            out += '  ' if grid[r, c] else '  '
            if (c+1) % n == 0 and c != nrow-1: # end of block
                out += '| '
        out += '\n'
        if (r+1) % n == 0 and r != nrow-1: # end of block
            out += ('-'*(n + 2*n))
            out += ('+' + '-'*(n + 2*n + 1)) *(n-1) + '\n'
    print(out)   

def model_sudoku(given):
    #empty cell
    e = 0
    # Dimensions of the sudoku problem
    ncol = nrow = len(given)
    n = int(ncol ** (1/2))

    # Model Variables
    cells = intvar(1,ncol,shape=given.shape,name="cells")
    
    # sudoku must start from initial clues
    facts = list(cells[given != e] == given[given != e])
    # rows constraint
    row_cons = [AllDifferent(row) for row in cells]
    # columns constraint
    col_cons = [AllDifferent(col) for col in cells.T]
    # blocks constraint
    # Constraints on blocks
    block_cons = []

    for i in range(0,nrow, n):
        for j in range(0,ncol, n):
            block_cons += [AllDifferent(cells[i:i+n, j:j+n])]

    return cells, facts, row_cons + col_cons + block_cons

def extract_solution(constraints, variables):
    solved = Model(constraints).solve()
    assert solved, "Model is unsatisfiable."     
    return variables.value()


# OCUS-based explanations of Unsatisfiability

OCUS, or Optimal Constrained Unsatisfiable SUbsets, is a cost-optimal unsatisfiable subset that has a structural constraint.
The OCUS algorithm takes advantage of the implicit hitting set duality to efficiently compute an OCUS. 

## OCUS without assumptions 

In [15]:
def OCUS(soft, soft_weights, hard=[], solver='ortools', verbose=1):
    ## Mip model
    if Model(hard+soft).solve():
        if verbose:
            print("Unexpectedly, the model is SAT")
        return []

    # decision variables of hitting set problem
    hs_vars = boolvar(shape=len(soft), name="hs_vars")

    hs_model = Model(
        # Objective: min sum(w_l * x_l)
        minimize=sum(soft_weights[id] * var for id, var in enumerate(hs_vars))
    )

    # instantiate hitting set solver
    hittingset_solver = SolverLookup.lookup(solver)(hs_model)

    while(True):
        # Get hitting set
        hittingset_solver.solve()
        hs_ids = np.where(hs_vars.value() == 1)[0]

        hs_soft = [soft[i] for i in hs_ids]

        # instantiate model every loop
        if not Model(hard+hs_soft).solve():
            return hs_soft

        # compute complement of model in formula F
        C = hs_vars[hs_vars.value() != 1]

        # Add complement as a new set to hit: sum x[j] * hij >= 1
        hittingset_solver += (sum(C) >= 1)

## OCUS with assumptions 

In [11]:
def OCUS_assum(soft, soft_weights, hard=[], solver='ortools', verbose=1):
    # init with hard constraints
    assum_model = Model(hard)

    # make assumption indicators, add reified constraints
    ind = BoolVar(shape=len(soft), name="ind")
    for i,bv in enumerate(ind):
        assum_model += [bv.implies(soft[i])]
    # to map indicator variable back to soft_constraints
    indmap = dict((v,i) for (i,v) in enumerate(ind))

    assum_solver = SolverLookup.lookup(solver)(assum_model)

    if assum_solver.solve(assumptions=ind):
        return []

    ## 
    hs_model = Model(
        # Objective: min sum(x_l * w_l)
        minimize=sum(x_l * w_l for x_l, w_l in zip(ind, soft_weights))
    )

    # instantiate hitting set solver
    hittingset_solver = SolverLookup.lookup(solver)(hs_model)

    while(True):
        hittingset_solver.solve()

        # Get hitting set
        hs = ind[ind.value() == 1]


        if not assum_solver.solve(assumptions=hs):
            return soft[ind.value() == 1]

        # compute complement of model in formula F
        C = ind[ind.value() != 1]

        # Add complement as a new set to hit: sum x[j] * hij >= 1
        hittingset_solver += (sum(C) >= 1)

# Testing the speed of OCUS with and without assumptions
## OCUS for unsat int model

In [13]:
model, weights = unsat_int_model()
# Number of times the OCUS is 
n = 100

# calculate total execution time
result_ocus = timeit.timeit(
    lambda: OCUS(soft=model.constraints, soft_weights=weights, hard=[]), 
    number=n
)
print(f"OCUS without assumptions:\n\tExecution time is {result_ocus / n} seconds")

result_ocus_assum = timeit.timeit(
    lambda: OCUS_assum(soft=np.array(model.constraints), soft_weights=weights, hard=[]), 
    number=n
)
print(f"OCUS with assumptions:\n\tExecution time is {result_ocus_assum / n} seconds")

OCUS without assumptions:
	Execution time is 0.0454317678399093 seconds
OCUS with assumptions:
	Execution time is 0.050682117760006806 seconds


## Explaining an unsatisfiable SUDOKU

In this part, we test the speed of solving a small unsatisfiable sudoku with and without assumptions.
We start from a given satisfiable sudoku, and solve it. 

To make the sudoku unsatisfiable, we select an empty cell at position (i,j) ***cell[i, j]*** with its corresponding target value **sol[i,j]** and add the inequality **[cell[i, j] != sol[i,j]]** to the constraints of the model.

For instance:

       3  | 4     
    4     |    2  
    ------+-------
    1     |    3  
       2  | 1     
       

We know that in the satisfying solution cell[0,0] must be value 2. To make our model unsatisfiable, we add  ___cell[0,0] != sol[0,0]___.

### Sudoku model

In [14]:
e = 0

given_4x4 = np.array([
    [ e, 3, 4, e ],
    [ 4, e, e, 2 ],
    [ 1, e, e, 3 ],
    [ e, 2, 1, e ],
])

given = given_4x4

display(HTML('<h3> INPUT SUDOKU</h3>'))
print_sudoku(given)

sudoku_vars, sudoku_facts, sudoku_constraints = model_sudoku(given)

sudoku_sol = extract_solution(
    constraints=sudoku_constraints + sudoku_facts, 
    variables=sudoku_vars
)

display(HTML('<h3> SOLUTION </h3>'))
print_sudoku(sudoku_sol)

   3  | 4     
4     |    2  
------+-------
1     |    3  
   2  | 1     



2  3  | 4  1  
4  1  | 3  2  
------+-------
1  4  | 2  3  
3  2  | 1  4  



### Speed of OCUS for SUDOKU

In [ ]:
all_cons = sudoku_constraints + list(sudoku_facts)
weights = [20] * len(sudoku_constraints) + [1] * (len(sudoku_facts) + 1)

unassigned_negated = [
    (var != sol_val) 
    for val, sol_val, var in zip(given.flatten(), sudoku_sol.flatten(), sudoku_vars.flatten())
    if val == e
]

n = len(unassigned_negated)

iter_ocus_negated = iter(unassigned_negated)

result_ocus = timeit.timeit(
    lambda: OCUS(soft=all_cons + [next(iter_ocus_negated)], soft_weights=weights, hard=[]), 
    number=n
)
print(f"OCUS without assumptions:\n\tExecution time is {result_ocus / n} seconds")

iter_ocus_ass_negated = iter(unassigned_negated)
result_ocus_assum = timeit.timeit(
    lambda: OCUS_assum(soft=np.array(all_cons  + [next(iter_ocus_ass_negated)]), soft_weights=weights, hard=[]),
    number=n
)
print(f"OCUS with assumptions:\n\tExecution time is {result_ocus_assum / n} seconds")